# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/satwindersingh/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Data Example

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [4]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Create Model**

In [5]:
from sklearn.linear_model import Ridge

In [6]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [7]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [8]:
y_pred = model.predict(X_test)

In [9]:
y_pred

array([15.34908128, 17.05755308, 12.73784965, 16.18231062, 10.85075815,
        9.87999576, 17.6105132 , 15.80786278, 11.32616781, 17.30158479,
       12.8883864 , 13.64670913, 13.71636726, 18.83377117, 17.38617584,
       11.59912699, 14.88899736, 10.07145317, 10.14692243, 17.90771073,
       10.25837266, 16.71492563, 20.57087744, 19.66643199, 10.14020781,
       13.40084066, 18.09910709, 10.80433113, 13.00876939, 13.79206361,
       12.73015096, 17.42108555, 11.50183684, 10.10362749, 16.18778637,
       10.45161746, 11.25953403, 10.42658319, 12.30681396, 11.82281519,
       14.75707677, 11.58372535, 12.01609545, 10.90016204, 12.55896716,
       11.62961585, 10.8495293 , 15.74187916, 14.09264772, 18.45114683,
       13.43419788, 14.05075373, 16.0980788 , 12.07046074, 13.15048011,
        8.75095421, 19.21013193, 12.92686996, 16.49277745, 14.83525505])

**Evaluation**

In [10]:
from sklearn.metrics import mean_squared_error

In [11]:
mean_squared_error(y_test,y_pred)

7.34177578903413

**Adjust Parameters and Re-evaluate**

In [22]:
model = Ridge(alpha=0.56)

In [23]:
model.fit(X_train,y_train)

Ridge(alpha=0.56)

In [24]:
y_pred = model.predict(X_test)

In [25]:
y_pred

array([15.73801575, 19.58315825, 11.46234626, 17.00143228,  9.18574206,
        7.03936645, 20.26230201, 17.28319474,  9.78924036, 19.20382657,
       12.40682936, 13.88719357, 13.72422206, 21.26641765, 18.41895497,
        9.99625142, 15.54551784,  7.71036551,  7.58014112, 20.37863332,
        7.81442364, 18.22749396, 24.64687547, 22.79344605,  7.99766391,
       12.66052877, 21.43350797,  8.08051602, 12.4278849 , 12.52273852,
       10.79962727, 19.2281913 , 10.08242685,  6.74275288, 17.30449917,
        7.79424134,  9.273121  ,  8.29725275, 10.59857948, 10.64571114,
       13.03531115,  9.78974746, 10.23322077,  8.07531131, 11.57551977,
       10.09724264,  9.0131756 , 16.25136684, 13.2511561 , 20.79357469,
       12.5071102 , 13.96710937, 17.54830596, 11.15181137, 12.56807871,
        5.53755514, 23.25169167, 12.62345749, 18.74887247, 15.18455502])

**Another Evaluation**

In [26]:
mean_squared_error(y_test,y_pred)

2.309830759753397

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [27]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

In [28]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################
from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [29]:
len(X_train)

140

In [30]:
len(X_eval)

30

In [31]:
len(X_test)

30

In [32]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [33]:
from sklearn.linear_model import Ridge

In [34]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [35]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [36]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [37]:
from sklearn.metrics import mean_squared_error

In [38]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823871

**Adjust Parameters and Re-evaluate**

In [43]:
model = Ridge(alpha=0.56)

In [44]:
model.fit(X_train,y_train)

Ridge(alpha=0.56)

In [45]:
y_eval_pred = model.predict(X_eval)

**Another Evaluation**

In [46]:
mean_squared_error(y_eval,y_eval_pred)

2.3715654421713235

**Final Evaluation (Can no longer edit parameters after this!)**

In [47]:
y_final_test_pred = model.predict(X_test)

In [48]:
mean_squared_error(y_test,y_final_test_pred)

2.24809607733547

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [49]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
model = Ridge(alpha=100)

In [53]:
from sklearn.model_selection import cross_val_score

In [54]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [55]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [56]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

8.215396464543607

**Adjust model based on metrics**

In [69]:
model = Ridge(alpha=0.56)

In [91]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [92]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.3439821853239087

**Final Evaluation (Can no longer edit parameters after this!)**

In [72]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=0.56)

In [73]:
y_final_test_pred = model.predict(X_test)

In [74]:
mean_squared_error(y_test,y_final_test_pred)

2.309830759753397

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [ ]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Ridge(alpha=100)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [ ]:
scores

In [ ]:
pd.DataFrame(scores)

In [ ]:
pd.DataFrame(scores).mean()

**Adjust model based on metrics**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [ ]:
pd.DataFrame(scores).mean()

**Final Evaluation (Can no longer edit parameters after this!)**

In [ ]:
# Need to fit the model first!
model.fit(X_train,y_train)

In [ ]:
y_final_test_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_final_test_pred)

----
----